In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

In [ ]:
import telebot #Biblioteca de criação do Bot Telegram
from re import sub
from time import sleep 
from os import walk, path
from datetime import date
from string import punctuation
from ipynb.fs.full.chamar import init
from ipynb.fs.full.pegaof import pega_ofertas
from dateutil.relativedelta import relativedelta
from ipynb.fs.full.xml2pdf_semforn import iniciapdf
from telebot.types import InlineKeyboardMarkup, InlineKeyboardButton #Biblioteca dos botões e markup


meses = []
nome_meses = ['JANEIRO', 'FEVEREIRO', 'MARÇO', 'ABRIL', 'MAIO', 'JUNHO', 'JULHO', 'AGOSTO', 'SETEMBRO', 'OUTUBRO', 'NOVEMBRO', 'DEZEMBRO']
now = date.today()

for i in range(3,-1,-1): #Calcula os 3 meses anteriores
    x = relativedelta(months=i)
    meses.append({nome_meses[((now - x).month)-1]:(now - x).month})
    
def junta_dici(ld):
    '''Função pra juntar uma lista de dicionários em um único'''
    resultado = {}
    for dici in ld:
        resultado.update(dici)
    return resultado

def junta_loja(pessoa):
    '''Função pra juntar as lojas contidas em um dicionário em uma única lista'''
    ljs = []
    if pessoa['lj1']:
        ljs.append('0')
    if pessoa['lj2']:
        ljs.append('1')
    if pessoa['lj3']:
        ljs.append('2') 
    if pessoa['lj4']:
        ljs.append('3')
    if pessoa['lj5']:
        ljs.append('4')
    if pessoa['lj6']:
        ljs.append('5')
    if pessoa['lj7']:
        ljs.append('6')
    return ljs
    
meses_junto = junta_dici(meses)
lis_dici = [{'ID':'0','FORN':'','NUMERO':0,'lj1':False,'lj2':False,'lj3':False,'lj4':False,'lj5':False,'lj6':False,'lj7':False,'pdf':False, 'mes':now.month}]

def insere_ld(id_,item, mes=0, forn=0, numero=0):
    '''Função pra inserir algo nas escolhas do usuário'''
    global lis_dici                  #Pega o dict de escolhas pelo global
    for i in lis_dici:
        if i['ID'] == id_:
            if numero == 1:
                i['NUMERO'] = item
                i['FORN'] = 0
                print(i)
                return True
            if numero and forn:
                raise ValueError
            if item == 'pdf':
                i['pdf'] = True
                return True
            if item ==  'preconf':
                i['pdf'] = False
                return True
            if not mes and not forn: #Se não mes e forn, o item mandado vira True
                i[item] = True
                return True
            elif mes:                #Se mes, declara o mes de preferencia dentro do dict de escolhas
                i['mes'] = item
                return True
            elif forn:              #Se forn, declara o forn de preferencia dentro do dict de escolhas
                i['FORN'] = sub('[^0-9]', '', item) #Deixa apenas os digitos
                i['NUMERO'] = 0
                return True

    lis_dici.append({'ID':id_,'lj1':False,'lj2':False,'lj3':False,'lj4':False,'lj5':False,'lj6':False,'lj7':False,'pdf':False, 'mes':now.month}) #Se não achar o ID ele cria um
    insere_ld(id_, item, mes, forn, numero) #E manda inserir oq foi pedido dentro do id criado

    
def limpa_ld(id_):
    '''zera as escolhas que não forem ID, MES ou FORN'''
    global lis_dici
    for i in lis_dici:
        if i['ID'] == id_:
            for k,v in i.items():
                if k != 'ID' and k != 'mes' and k != 'FORN':
                    i[k] = False
    

def pega_escolhas(id_):
    '''Função pra retornar o dict de escolhas a partir de um ID'''
    global lis_dici
    for i in lis_dici:
        if i['ID'] == id_:
            return i
    
    
def uma_linha(nome_arquivo):
    '''Função pra abrir arquivos de texto e retornar a primeira linha'''
    with open (f'C:/Users/pdv/Desktop/gabriel/{nome_arquivo}', 'r') as arquivo:
        for linha in arquivo:
            return linha        

            
chaveapi = uma_linha('chaveapitelebot.txt')    #Pega a chave API do bot de um arquivo local  
bot = telebot.TeleBot(chaveapi) #Cria uma instância do bot


def checkforn(forn):
    '''Função que devolve uma lista de fornecedores (e cnpjs) a partir de uma string'''
    cnpjs, nomes, cnpj = [], [], ''
    with open("cnpjs.txt", 'r') as cads:
        for linha in cads:
            if forn.lower() in linha.lower():
                cnpj = ''.join([i for i in linha[-19:-9] if i not in punctuation])
                cnpjs.append(cnpj)
                nomes.append(linha[0:-19])
        return (cnpjs, nomes)
            
            
def faz_botao(dicionario): 
    '''Função pra criar os botões de resposta'''
    markup = InlineKeyboardMarkup() #inicia o markup (marcação)
    for botao, retorno in dicionario.items(): #Desempacota os botões e seus retornos do dicionário
        markup.add(InlineKeyboardButton(str(botao), callback_data=str(retorno))) #cria o botão dentro do markup
    return markup #retorna o markup
    
    
def poezero(num):
    '''Função pra pegar um mes e se for menor que 10 coloca o zero'''
    try:
        num = int(num)
    except:
        print('erro no poezero')
    mes = ''
    if num <= 9:
        mes = f'0{str(num)}'
    else:
        mes = str(num)
    return mes
    

def envia(mensagemID, pdf = 0):
    '''Função para enviar documentos ou fotos pro usuário'''
    pasta = fr'C:\Users\pdv\projetos_jupyter\retorno\\{mensagemID}'
    if pdf:
        pasta = fr'C:\Users\pdv\Desktop\selenium-imgs\pdfs\\{mensagemID}'
    for diretorio, subpastas, arquivos in walk(pasta):
        if not arquivos:
            text = 'Desculpe, não achei notas desse fornecedor nessa loja'
            bot.send_message(mensagemID,text)
            return 0
        for arquivo in arquivos:
            x = path.join(pasta, arquivo)
            if pdf:
                file = open(x, 'rb')
                bot.send_document(mensagemID, file)
                file.close()
            else:
                photo = open(x,'rb')
                bot.send_photo(mensagemID, photo=photo)
                photo.close()
    bot.reply_to(mensagem, 'pronto ;)') #Mensagem pra mostrar que deu certo
    
    
def pega_ofertas():
    pastas = {
        'Dalbem': r'C:\Users\pdv\Desktop\selenium-imgs\Dalbem\feito',
        'Fartura': r'C:\Users\pdv\Desktop\selenium-imgs\Fartura',
        'Goodbom': r'C:\Users\pdv\Desktop\selenium-imgs\goodbom',
        'Oba': r'C:\Users\pdv\Desktop\selenium-imgs\oba',
        'Pague Menos': r'C:\Users\pdv\Desktop\selenium-imgs\PagueMenos',       
    }
    dict_ofertas = {}
    for merc,pasta in pastas.items():
        for diretorio, subpastas, arquivos in walk(pasta):
            for arquivo in arquivos:
                dict_ofertas[merc] = f'ofertas: {pasta}'
                break
    return dict_ofertas


def envia_ofertas(mensagemID, pasta):
    '''Função para enviar ofertas pro usuário'''
    for diretorio, subpastas, arquivos in walk(pasta):
        for arquivo in arquivos:
            x = path.join(pasta, arquivo)
            file = open(x, 'rb')
            if 'pdf' in arquivo:
                bot.send_document(mensagemID, file)
            else:
                bot.send_photo(mensagemID, file)
            file.close()
    
    
@bot.message_handler(commands=["ofertas"])
def ofertas(mensagem): 
    mensagemID = mensagem.chat.id
    bot.send_message(mensagemID,'Qual mercado?' , reply_markup = faz_botao(pega_ofertas()))
    
    
@bot.message_handler(commands=["numero"])
def numero(mensagem):
    '''Função que gerencia o /numero'''
    id_ = mensagem.chat.id #O id é uma parte da estrutura aninhada da mensagem ({'content_type': 'text', 'id': 9, 'message_id': 9,'from_user': {'id': 945441763, 'is_bot': False, 'first_name': 'Gbr', 'username': 'Gbrdoidao', 'last_name': 'Doidao', 'language_code': 'pt-br', 'can_join_groups': None, 'can_read_all_group_messages': None, 'supports_inline_queries': None}, 'date': 1649940793,'chat': {'id': 945441763, 'type': 'private', 'title': None, 'username': 'Gbrdoidao', 'first_name': 'Gbr', 'last_name': 'Doidao', 'photo': None, 'bio': None, 'description': None, 'invite_link': None, 'pinned_message': None, 'permissions': None, 'slow_mode_delay': None, 'message_auto_delete_time': None, 'sticker_set_name': None, 'can_set_sticker_set': None, 'linked_chat_id': None, 'location': None},'sender_chat': None, 'forward_from': None, 'forward_from_chat': None, 'forward_from_message_id': None, 'forward_signature': None, 'forward_sender_name': None, 'forward_date': None, 'reply_to_message': None, 'via_bot': None, 'edit_date': None, 'media_group_id': None, 'author_signature': None, 'text': 'ao', 'entities': None, 'caption_entities': None, 'audio': None, 'document': None, 'photo': None, 'sticker': None, 'video': None, 'video_note': None, 'voice': None, 'caption': None, 'contact': None, 'location': None, 'venue': None, 'animation': None, 'dice': None, 'new_chat_member': None, 'new_chat_members': None, 'left_chat_member': None, 'new_chat_title': None, 'new_chat_photo': None, 'delete_chat_photo': None, 'group_chat_created': None, 'supergroup_chat_created': None, 'channel_chat_created': None, 'migrate_to_chat_id': None, 'migrate_from_chat_id': None, 'pinned_message': None, 'invoice': None, 'successful_payment': None, 'connected_website': None, 'reply_markup': None, 'json': {'message_id': 9, 'from': {'id': 945441763, 'is_bot': False, 'first_name': 'Gbr', 'last_name': 'Doidao', 'username': 'Gbrdoidao', 'language_code': 'pt-br'}, 'chat': {'id': 945441763, 'first_name': 'Gbr', 'last_name': 'Doidao', 'username': 'Gbrdoidao', 'type': 'private'}, 'date': 1649940793, 'text': 'ao'}})
    if len(mensagem.text) < 11:
        bot.send_message(id_, 'preciso de mais números')
        return
    global meses_junto
    numero = mensagem.text[8:] #Pega a parte importante da mensagem (sem o /numero )
    insere_ld(id_, item=numero, numero=1)
    print(mensagem.from_user.username, mensagem.from_user.first_name, ' pesquisou notas n° ', numero)
    dicio_pdf = {'Preço Notas':'numero preconf','Nota em PDF':'numero pdf'} #Botão pdf / preço {'nome_botão':'retorno do botão'}
#     dicio_lojas = {'LJ1 - Guanabara':'lj1' ,'LJ2 - Primavera':'lj2','LJ3 - Iguatemi':'lj3','LJ4 - Flamboyant':'lj4', 'LJ5 - São Paulo':'lj5',
#                    'LJ6 - Valinhos':'lj6', 'LJ7 - Ouro Verde':'lj7', 'ESCOLHIDO':'escolheu'} #Botões das lojas e seus retornos 
    bot.send_message(id_, ' Qual Mes? ', reply_markup=faz_botao(meses_junto)) #Botao pro user escolher o mes da nota
    bot.send_message(id_, ' Qual vc prefere? ' ,reply_markup=faz_botao(dicio_pdf)) #Cria o botão pro escolher entre PDF e meu preço
#     bot.send_message(id_, ' LOJAS: ' ,reply_markup=faz_botao(dicio_lojas)) #Cria o botão pro escolher entre PDF e meu preço
    
    
@bot.message_handler(commands=["notas"])
def notas(mensagem):
    '''Função que gerencia o /notas'''
    global meses_junto
    id_ = mensagem.chat.id #O id é uma parte da estrutura aninhada da mensagem ({'content_type': 'text', 'id': 9, 'message_id': 9,'from_user': {'id': 945441763, 'is_bot': False, 'first_name': 'Gbr', 'username': 'Gbrdoidao', 'last_name': 'Doidao', 'language_code': 'pt-br', 'can_join_groups': None, 'can_read_all_group_messages': None, 'supports_inline_queries': None}, 'date': 1649940793,'chat': {'id': 945441763, 'type': 'private', 'title': None, 'username': 'Gbrdoidao', 'first_name': 'Gbr', 'last_name': 'Doidao', 'photo': None, 'bio': None, 'description': None, 'invite_link': None, 'pinned_message': None, 'permissions': None, 'slow_mode_delay': None, 'message_auto_delete_time': None, 'sticker_set_name': None, 'can_set_sticker_set': None, 'linked_chat_id': None, 'location': None},'sender_chat': None, 'forward_from': None, 'forward_from_chat': None, 'forward_from_message_id': None, 'forward_signature': None, 'forward_sender_name': None, 'forward_date': None, 'reply_to_message': None, 'via_bot': None, 'edit_date': None, 'media_group_id': None, 'author_signature': None, 'text': 'ao', 'entities': None, 'caption_entities': None, 'audio': None, 'document': None, 'photo': None, 'sticker': None, 'video': None, 'video_note': None, 'voice': None, 'caption': None, 'contact': None, 'location': None, 'venue': None, 'animation': None, 'dice': None, 'new_chat_member': None, 'new_chat_members': None, 'left_chat_member': None, 'new_chat_title': None, 'new_chat_photo': None, 'delete_chat_photo': None, 'group_chat_created': None, 'supergroup_chat_created': None, 'channel_chat_created': None, 'migrate_to_chat_id': None, 'migrate_from_chat_id': None, 'pinned_message': None, 'invoice': None, 'successful_payment': None, 'connected_website': None, 'reply_markup': None, 'json': {'message_id': 9, 'from': {'id': 945441763, 'is_bot': False, 'first_name': 'Gbr', 'last_name': 'Doidao', 'username': 'Gbrdoidao', 'language_code': 'pt-br'}, 'chat': {'id': 945441763, 'first_name': 'Gbr', 'last_name': 'Doidao', 'username': 'Gbrdoidao', 'type': 'private'}, 'date': 1649940793, 'text': 'ao'}})
    enviado = mensagem.text[7:] #Pega a parte importante da mensagem (sem o /notas )
    fornecedor = checkforn(enviado)
    print('resultado checkforn: ', fornecedor)
    if not fornecedor[0]:
        bot.reply_to(mensagem, 'Fornecedor nâo encontrado') #Mensagem pra mostrar que deu errado a busca de forn
        return 0
    if len(fornecedor[0]) > 1:
        dicio_forn = {}
        for cnpj in fornecedor[0]:
            for empresa in fornecedor[1]:   
                dicio_forn[empresa] = f'forn: {cnpj}'
        bot.send_message(id_, 'Qual Fornecedor?', reply_markup=faz_botao(dicio_forn))
    else:
        insere_ld(id_, fornecedor[0][0], forn=1) #Coloca o fornecedor em escolhas (dicionario do user)

    print(mensagem.from_user.username, mensagem.from_user.first_name, ' pesquisou notas do ', fornecedor)
    dicio_pdf = {'Preço Notas':'preconf','Nota em PDF':'pdf'} #Botão pdf / preço {'nome_botão':'retorno do botão'}
    dicio_lojas = {'LJ1 - Guanabara':'lj1' ,'LJ2 - Primavera':'lj2','LJ3 - Iguatemi':'lj3','LJ4 - Flamboyant':'lj4', 'LJ5 - São Paulo':'lj5',
                   'LJ6 - Valinhos':'lj6', 'LJ7 - Ouro Verde':'lj7', 'ESCOLHIDO':'escolheu'} #Botões das lojas e seus retornos 
    bot.send_message(id_, ' Qual Mes? ', reply_markup=faz_botao(meses_junto)) #Botao pro user escolher o mes da nota
    bot.send_message(id_, ' Qual vc prefere? ' ,reply_markup=faz_botao(dicio_pdf)) #Cria o botão pro escolher entre PDF e meu preço
    bot.send_message(id_, ' LOJAS: ' ,reply_markup=faz_botao(dicio_lojas)) #Cria o botão pro escolher entre PDF e meu preço
    

@bot.message_handler(func=lambda message: True) #Decorador que cuida de mensagens (não comandos)
def resposta(mensagem):
    '''Função que cuida de mensagens sem comandos (/) '''
    texto = '''Olá! Seja bem vindo ao Fartura Bot  
    Estes são os meus Comandos:
    
    /notas <Nome Fornecedor>
    
    /ofertas
    
    =-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=''' #Mensagem padrão de resposta
    bot.reply_to(mensagem, texto) #Responde a mensagem dele com o texto padrão
    
    
@bot.callback_query_handler(func=lambda call: True)
def handle_query(chamado):
    '''Função pra cuidar dos retornos (callbacks) do KeyboardButtons'''
    id_ = chamado.message.chat.id
    retorno = chamado.data
    inteiro = False
    num = 0
    try:
        inteiro = int(retorno)
    except:
        pass
    if retorno.startswith('lj') or retorno == 'pdf' or retorno == 'preconf':
        insere_ld(id_,retorno)
    elif retorno.startswith('numero'):
        insere_ld(id_,item=retorno[7:])
        num = 1
        print('escolhido foi')
    elif retorno.startswith('ofertas: '):
        envia_ofertas(id_, retorno[9:])
    elif inteiro and inteiro in range(13): #Pega o retorno dos botões de meses
        insere_ld(id_, retorno, mes=1)
    elif retorno.startswith('forn:'):
        insere_ld(id_, retorno, forn=1)
    if retorno == 'escolheu' or num == 1:
        print('to no escolhido')
        escolhas = pega_escolhas(id_)
        lojas = junta_loja(escolhas)
        mes = poezero(escolhas['mes'])
        pdf = escolhas['pdf']
        forn = escolhas['FORN']
        numero = escolhas['NUMERO']
        if pdf:
            if num == 0:
                r = iniciapdf(id1 = str(id_), fornecedor=forn, numero=numero, lojas=lojas, mes=mes) #Chama a função de criar pdf
            if num == 1:
                r = iniciapdf(id1 = str(id_),numero=numero, mes=mes) #Chama a função de criar pdf
            envia(id_, pdf=1)                
        else:
            if num == 0:
                r = init(ean = forn, mes = mes, loja = lojas, idz = str(id_), numero=numero) #Chama a função de criar dataframes dos preços
            if num == 1:
                r = init(idz = str(id_), numero=numero, mes = mes) #Chama a função de criar dataframes dos preços
            envia(id_, pdf=0)                
        limpa_ld(id_)

    
while True:
    #Esse while é o que mantém o código sempre rodando
    try:
        bot.polling(none_stop=True, interval=0, timeout=0) #Mantém o bot rodando infinitamente
    except:
        sleep(10) #se der erro ele espera 10 segundos


['35220406204131001904550010008559891479222245.xml', '35220406204131001904550010008614101217181442.xml']


2022-04-30 09:17:37,540 (__init__.py:667 MainThread) ERROR - TeleBot: "A request to the Telegram API was unsuccessful. Error code: 409. Description: Conflict: terminated by other getUpdates request; make sure that only one bot instance is running"


{'ID': 945441763, 'lj1': False, 'lj2': False, 'lj3': False, 'lj4': False, 'lj5': False, 'lj6': False, 'lj7': False, 'pdf': False, 'mes': 4, 'NUMERO': '199', 'FORN': 0}
Gbrdoidao Gbr  pesquisou notas n°  199


2022-04-30 09:17:41,483 (__init__.py:667 MainThread) ERROR - TeleBot: "A request to the Telegram API was unsuccessful. Error code: 409. Description: Conflict: terminated by other getUpdates request; make sure that only one bot instance is running"


escolhido foi
to no escolhido
entrei init
000015487
000014555
000086638
000087492
000002390
000318375
000009906
000009946
000146199
004153236
002162524
001647815
000176199
000015340
000015477
000015482
000007384
001628656
001739806
000348504
000348505
000727258
000010011
000010064
000765885
000046489
000086705
000086706
000002199
000195689
000103045
012230334
001276558
000078439
001658395
000607742
000348506
000348507
000348508
000199866
003119875
000768139
000007426
000086636
000087491
000002167
000348929
002681642
000000081
000519630
001276559
000191804
000013079
000015338
000015493
000074389
000348509
000348510
000199867
000009994
000001765
000086637
000087493
000002137
000002203
000216603
000009926
004154687
000408021
000577776
001177800
000053605
000053716
000053850
000053984
000199650
000018538
000015339
000015481
000024207
000173199
000819920
000084801
000005298
000010121
000002500
000513617
001153009
000833688
000155192
004125260
000097124
000275473
000199461
000350897
01219940

...

Escolha foi  un
qcx =  0
Escolha foi  un
qcx =  0
Escolha foi  un
qcx =  0
Escolha foi  un
qcx =  0
Escolha foi  un
qcx =  0
Escolha foi  un
qcx =  0
Escolha foi  un
qcx =  0
Escolha foi  un
qcx =  0
Escolha foi  un
qcx =  0
Escolha foi  un
qcx =  0
Escolha foi  un
qcx =  0
Escolha foi  un
qcx =  0
Escolha foi  maluco
qcx =  0
Escolha foi  ('ean', 12)
qcx =  12 

Escolha foi  ('ean', 20)
qcx =  20 

Escolha foi  ('ean', 20)
qcx =  20 

Escolha foi  ('ean', 10)
qcx =  10 

Escolha foi  ('ean', 10)
qcx =  10 

Escolha foi  ('ean', 30)
qcx =  30 

Escolha foi  ('ean', 30)
qcx =  30 

Escolha foi  ('ean', 6)
qcx =  6 

Escolha foi  ('ean', 6)
qcx =  6 

Escolha foi  ('ean', 12)
qcx =  12 

Escolha foi  un
qcx =  0
Escolha foi  un
qcx =  0
Escolha foi  un
qcx =  0
Escolha foi  un
qcx =  0
Escolha foi  un
qcx =  0
Escolha foi  un
qcx =  0
Escolha foi  un
qcx =  0
Escolha foi  un
qcx =  0
Escolha foi  un
qcx =  20 

Escolha foi  un
qcx =  20 

Escolha foi  un
qcx =  12 

Escolha foi  un
qcx =

2022-04-30 09:18:18,189 (__init__.py:667 MainThread) ERROR - TeleBot: "A request to the Telegram API was unsuccessful. Error code: 409. Description: Conflict: terminated by other getUpdates request; make sure that only one bot instance is running"


Escolha foi  un
qcx =  0
Escolha foi  un
qcx =  0
Escolha foi  un
qcx =  0
Escolha foi  un
qcx =  0
Escolha foi  un
qcx =  0
Escolha foi  un
qcx =  0
Escolha foi  un
qcx =  0
Escolha foi  un
qcx =  0
Escolha foi  un
qcx =  0
Escolha foi  un
qcx =  0
Escolha foi  un
qcx =  0
Escolha foi  un
qcx =  0
Escolha foi  maluco
qcx =  0
Escolha foi  maluco
qcx =  0
Escolha foi  maluco
qcx =  0
Escolha foi  maluco
qcx =  0
Escolha foi  maluco
qcx =  0
Escolha foi  maluco
qcx =  0
Escolha foi  ('ean', 12)
qcx =  12 

Escolha foi  ('ean', 20)
qcx =  20 

Escolha foi  ('ean', 20)
qcx =  20 

Escolha foi  ('ean', 20)
qcx =  20 

Escolha foi  ('ean', 20)
qcx =  20 

Escolha foi  ('ean', 10)
qcx =  10 

Escolha foi  ('ean', 30)
qcx =  30 

Escolha foi  maluco
qcx =  0
Escolha foi  ('ean', 20)
qcx =  20 

Escolha foi  ('ean', 12)
qcx =  12 

Escolha foi  ('ean', 12)
qcx =  12 

Escolha foi  un
qcx =  0
Escolha foi  un
qcx =  0
Escolha foi  un
qcx =  0
Escolha foi  un
qcx =  0
Escolha foi  un
qcx =  0
Es

2022-04-30 09:18:38,630 (__init__.py:667 MainThread) ERROR - TeleBot: "A request to the Telegram API was unsuccessful. Error code: 409. Description: Conflict: terminated by other getUpdates request; make sure that only one bot instance is running"


Escolha foi  un
qcx =  12 

Escolha foi  un
qcx =  0
Escolha foi  un
qcx =  6 

Escolha foi  un
qcx =  24 

Escolha foi  un
qcx =  24 

Escolha foi  un
qcx =  24 

Escolha foi  un
qcx =  24 

Escolha foi  un
qcx =  6 

Escolha foi  un
qcx =  48 

Escolha foi  un
qcx =  36 

Escolha foi  un
qcx =  32 

Escolha foi  un
qcx =  12 

Escolha foi  un
qcx =  12 

Escolha foi  un
qcx =  12 

Escolha foi  maluco
qcx =  0
Escolha foi  maluco
qcx =  0
Escolha foi  maluco
qcx =  0
Escolha foi  maluco
qcx =  0
Escolha foi  maluco
qcx =  0
Escolha foi  un
qcx =  0
Escolha foi  un
qcx =  6 

Escolha foi  un
qcx =  0
Escolha foi  un
qcx =  0
Escolha foi  un
qcx =  0
Escolha foi  ('ean', 30)
qcx =  30 

Escolha foi  ('ean', 6)
qcx =  6 

Escolha foi  ('ean', 30)
qcx =  30 

Escolha foi  ('ean', 30)
qcx =  30 

Escolha foi  ('ean', 30)
qcx =  30 

Escolha foi  ('ean', 15)
qcx =  15 

Escolha foi  ('ean', 6)
qcx =  6 

Escolha foi  un
qcx =  0
Escolha foi  un
qcx =  0
Escolha foi  un
qcx =  0
Escolha foi

In [ ]:
'''import time

def calcula_tempo(funcao):
    def wrapper(*args, **kwargs):     
        tempo_comeco = time.time()
        x = funcao(*args, **kwargs)
        tempo_final = time.time()
        print(f'sua função demorou {tempo_final - tempo_comeco}')
        return x
    return wrapper
        
        
l1 = list(range(-200,4000))
l2 = list(range(3000))
l3 = []

@calcula_tempo
def checa_lista(l1,l2,l3):
    for i in l1:
        if i not in l2:
            l3.append(i)
    return l3

print(checa_lista(l1,l2,l3))

@calcula_tempo
def Diff(l1, l2):
    return list(set(l1) - set(l2)) + list(set(l2) - set(l1))

print(Diff(l1,l2))'''

In [5]:
pip install schedule

  Using cached schedule-1.1.0-py2.py3-none-any.whl (10 kB)
Note: you may need to restart the kernel to use updated packages.
